In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/vseros-ai-dzen/test (8).csv
/kaggle/input/vseros-ai-dzen/items.parquet
/kaggle/input/vseros-ai-dzen/train (12).csv
/kaggle/input/vseros-ai-dzen/sample_submission (5).csv


# Сиды

In [2]:
import numpy as np
import pandas as pd
import torch
import os
import random

seed=42

os.environ['PYTHONHASHSEED']=str(seed)
np.random.seed(seed)
random.seed(seed)
torch.cuda.manual_seed(seed)
torch.cuda.manual_seed_all(seed)
torch.manual_seed(seed)

In [3]:
#device

# Импорты

In [4]:
from transformers import AutoModelForSequenceClassification, AutoTokenizer, TrainingArguments, Trainer
from torch.utils.data import Dataset, DataLoader
from sklearn.model_selection import train_test_split
from sklearn.metrics import f1_score


2025-09-18 20:03:28.631681: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:477] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1758225808.833338      36 cuda_dnn.cc:8310] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1758225808.891398      36 cuda_blas.cc:1418] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered


# TEST MODE

In [5]:
TEST_MODE=True

# Загрузка данных

In [6]:
items=pd.read_parquet('/kaggle/input/vseros-ai-dzen/items.parquet')
train=pd.read_csv('/kaggle/input/vseros-ai-dzen/train (12).csv')
test=pd.read_csv('/kaggle/input/vseros-ai-dzen/test (8).csv')
sample=pd.read_csv('/kaggle/input/vseros-ai-dzen/sample_submission (5).csv')

## text_target2num

In [7]:
#text_target2num={lg: i for i, lg in enumerate(sorted(train['target'].unique()))}
#num2text_target={i: lg for lg, i in text_target2num.items()}

In [8]:
'''# делаем стабильный маппинг только из TRAIN
classes = sorted(pairs_train["target"].dropna().unique().tolist())
cls2id = {c:i for i,c in enumerate(classes)}
id2cls = {i:c for c,i in cls2id.items()}
num_labels = len(classes)

train = train[train["target"].isin(classes)].copy()
train["target"] = train["target"].map(cls2id).astype(int)

# safety: никаких NaN/вылетов
assert train["label_id"].between(0, num_labels-1).all()
'''

'# делаем стабильный маппинг только из TRAIN\nclasses = sorted(pairs_train["target"].dropna().unique().tolist())\ncls2id = {c:i for i,c in enumerate(classes)}\nid2cls = {i:c for c,i in cls2id.items()}\nnum_labels = len(classes)\n\ntrain = train[train["target"].isin(classes)].copy()\ntrain["target"] = train["target"].map(cls2id).astype(int)\n\n# safety: никаких NaN/вылетов\nassert train["label_id"].between(0, num_labels-1).all()\n'

In [9]:
items

,authorId,content,itemId,title
0,22062724202903043,Британский кот — олицетворение невозмутимости....,1155255918526217609,"Как воспитать британского кота добрым, ласковы..."
1,5021973993129217102,Комнатные растения можно встретить в квартирах...,3466678402942852201,Вызывают судороги и смерть: пять опасных комна...
2,5843309594367696221,КОНТЕЙНЕР И БЫТОВКА Бывший в употреблении морс...,9802178366207388656,Из чего построить дачный дом? Семь доступных в...
3,4785891415385376217,Советский разведчик Аркадий Корнев с гранатой ...,5959787242217250844,Как осажденные советские солдаты в Афганистане...
4,6405657875686126019,Преимущества и недостатки оснастки и техники...,14212405135790401725,Преимущества и недостатки оснастки и техники д...
...,...,...,...,...
249446,12989058333369026839,Писатель и блогер Эрик Баркер о кризисе средне...,5627883028155472938,Выключить Сизифа: чем заменить бессмысленные п...
249447,22062724202903043,"Крупная, страшная и одна из самых опасных. Габ...",3232361825780066533,Как живёт белая акула: 9 интересных привычек и...
249448,2558767161391265740,Фото моделей сарафана-комбинезона и рисунки ле...,5023376663567542850,Простые для начинающих бохо выкройки сарафанов...
249449,8948519061459915922,"Много всяких-разных групп было в 90-х, но сего...",1917052039832459661,"""Популярные, но не очень"". И такие поп-группы ..."


In [10]:
train

,leftItemId,rightItemId,target
0,8456188648456666255,17842617327504664637,relevant_minus
1,2587705644018673303,4730919752548875048,relevant_minus
2,14440570074797857092,9007102992088629866,relevant
3,10768915202093280757,8382562539112058142,relevant
4,2799154957886243971,11333177839206244806,no_relevant
...,...,...,...
211388,1009839069153283157,6611620544658570875,relevant_minus
211389,6699347851365010050,8501386455539940627,relevant
211390,9512080461790124363,10345480721077553175,relevant_minus
211391,14191690305155376981,18355060322586231017,no_relevant


# Merge

In [11]:
import pandas as pd

# 1) Load
items = pd.read_parquet("/kaggle/input/vseros-ai-dzen/items.parquet")[["itemId","authorId","title","content"]]
pairs_train = pd.read_csv("/kaggle/input/vseros-ai-dzen/train (12).csv", dtype={"leftItemId":str, "rightItemId":str, "target":str})






if TEST_MODE:
    pairs_train=pairs_train[:55000]
else:
    pairs_train=pairs_train





# 2) Make sure keys are strings (protect against overflow)
items["itemId"] = items["itemId"].astype(str)

# 3) Deduplicate items if needed
items = items.drop_duplicates("itemId", keep="last").set_index("itemId")

# 4) Prepare left/right views and join
left_meta  = items.rename(columns=lambda c: f"left_{c}")    # columns: left_authorId,left_title,left_content
right_meta = items.rename(columns=lambda c: f"right_{c}")   # columns: right_authorId,right_title,right_content

train = (pairs_train
      .join(left_meta,  on="leftItemId")
      .join(right_meta, on="rightItemId"))

# (optional) keep a tidy column order
cols = ["left_title","right_title" ,"target",
        ]
train = train[cols]
#train['target']=train['target'].map(text_target2num)


In [12]:
pairs_test = pd.read_csv("/kaggle/input/vseros-ai-dzen/test (8).csv", dtype={"leftItemId":str, "rightItemId":str, "target":str})

test = (pairs_test
      .join(left_meta,  on="leftItemId")
      .join(right_meta, on="rightItemId"))

# (optional) keep a tidy column order
cols = ["left_title","right_title" ,
        ]
test = test[cols]

#test['target']=test['target'].map(text_target2num)


# Маппинг

In [13]:
# 1) Чистим таргеты до 4-классовой схемы
WANTED = ['no_relevant','relevant_minus','relevant','relevant_plus', 'doubles']

train = train[train['target'].isin(WANTED)].copy()

# 2) Фиксированный маппинг (сохраняет "порядок" классов)
cls2id  = {c:i for i,c in enumerate(WANTED)}   # {'no_relevant':0, 'relevant_minus':1, 'relevant':2, 'relevant_plus':3}
id2cls  = {i:c for c,i in cls2id.items()}
num_labels = len(WANTED)  # 4

train['target'] = train['target'].map(cls2id).astype('int64')

# sanity
#assert set(train['target'].unique()) == set(WANTED)
#assert train['target'].between(0, num_labels-1).all()

# 3) Стратифицированный сплит


In [14]:
train['target'].unique()

array([1, 2, 0, 3])

In [15]:
train_data, eval_data = train_test_split(
    train, test_size=0.2, stratify=train['target'], random_state=seed
)

# Dataset

In [16]:
class DzenDataset(Dataset):
    def __init__(self, df, tokenizer, max_len=128, has_label=True):
        self.df = df.reset_index(drop=True)
        self.tokenizer = tokenizer
        self.max_len = max_len
        self.has_label = has_label

    def __len__(self):
        return len(self.df)

    def __getitem__(self, idx):
        r = self.df.iloc[idx]

        enc = self.tokenizer(
            r["left_title"] or "",
            r["right_title"] or "",
            max_length=self.max_len,
            truncation=True,
            padding="max_length",
            return_tensors="pt"
        )

        item = {
            "input_ids": enc["input_ids"].squeeze(0),
            "attention_mask": enc["attention_mask"].squeeze(0),
        }
        # add token_type_ids for BERT-like models if present
        if "token_type_ids" in enc:
            item["token_type_ids"] = enc["token_type_ids"].squeeze(0)

        if self.has_label:
            item["labels"] = torch.tensor(int(r["target"]), dtype=torch.long)

        return item


# Model

In [17]:
from sentence_transformers import CrossEncoder

# Инициализация CrossEncoder модели
model = AutoModelForSequenceClassification.from_pretrained('xlm-roberta-large', num_labels=5,)

config.json:   0%|          | 0.00/697 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/1.44G [00:00<?, ?B/s]

Some weights of XLMRobertaForSequenceClassification were not initialized from the model checkpoint at deepvk/USER-bge-m3 and are newly initialized: ['classifier.dense.bias', 'classifier.dense.weight', 'classifier.out_proj.bias', 'classifier.out_proj.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [ ]:
#xlm-roberta-large deepvk/USER-bge-m3

# Tokenizer

In [18]:
tokenizer=AutoTokenizer.from_pretrained('xlm-roberta-large')
#ai-forever/FRIDA

tokenizer_config.json: 0.00B [00:00, ?B/s]

sentencepiece.bpe.model:   0%|          | 0.00/1.04M [00:00<?, ?B/s]

tokenizer.json: 0.00B [00:00, ?B/s]

special_tokens_map.json:   0%|          | 0.00/963 [00:00<?, ?B/s]

# Создание датасетов

In [19]:
train_dataset=DzenDataset(train_data, tokenizer, max_len=128, has_label=True )
eval_dataset=DzenDataset(eval_data, tokenizer, max_len=128, has_label=True)

test_dataset=DzenDataset(test, tokenizer, max_len=128,  has_label=False)


# TrainingArguments

In [20]:
from sentence_transformers import SentenceTransformerTrainingArguments, SentenceTransformerTrainer

In [21]:
args=TrainingArguments(
    output_dir='goal',
    per_device_train_batch_size=32,
    per_device_eval_batch_size=32,
    num_train_epochs=5,
    learning_rate=3e-6,
    lr_scheduler_type='cosine',
    eval_strategy='epoch',
    report_to='none',
    #logging_strategy="steps",
    #save_strategy="epoch",
    load_best_model_at_end=True,
    metric_for_best_model="f1_weighted",
    weight_decay=0.02,
    warmup_ratio=0.06,
    fp16=True,
    #max_grad_norm=1.0,
    label_smoothing_factor=0.025
)

# Compute Metrics

In [22]:
def compute_metrics(eval_pred):
    logits, labels = eval_pred
    preds = logits.argmax(axis=-1)
    return {
        "f1_weighted": f1_score(labels, preds, average="weighted", zero_division=0),
        "f1_macro":    f1_score(labels, preds, average="macro",    zero_division=0),
        "accuracy":    (preds == labels).mean(),
    }


# Trainer

In [23]:
trainer=Trainer(
    model=model,
    args=args,
    train_dataset=train_dataset,
    eval_dataset=eval_dataset,
    tokenizer=tokenizer,
    compute_metrics=compute_metrics,
    
)

/tmp/ipykernel_36/1803628416.py:1: FutureWarning: `tokenizer` is deprecated and will be removed in version 5.0.0 for `Trainer.__init__`. Use `processing_class` instead.
  trainer=Trainer(


# Train

In [24]:
trainer.train()

Epoch,Training Loss,Validation Loss,F1 Weighted,F1 Macro,Accuracy
1,No log,1.175485,0.456231,0.439981,0.468979
2,1.185300,1.138341,0.513427,0.502402,0.515827
3,0.986200,1.225350,0.512597,0.506884,0.513295


TrainOutput(global_step=1482, training_loss=0.9947132723212403, metrics={'train_runtime': 1953.4521, 'train_samples_per_second': 24.254, 'train_steps_per_second': 0.759, 'total_flos': 1.1038568609276928e+16, 'train_loss': 0.9947132723212403, 'epoch': 3.0})

# Submission

In [25]:
pred= trainer.predict(test_dataset)
test_pred_ids = pred.predictions.argmax(-1)
test_pred_text = [id2cls[i] for i in test_pred_ids]

submission = sample.copy()
submission["target"] = test_pred_text
submission.to_csv("submission4.csv", index=False)